# Taxonomy

## Dataset preparation

There are three subsets of data. They correspond to filter by title, body, and tags. 
1. posts_by_title: Posts containing "Github Actions" or its variants in the question title (github actions, github-action, Github actions, Github-actions, Github Actions, Github-Actions).
2. posts_by_body: Posts containing "Github Actions" or its variants in the question body.
3. posts_by_tags: Posts tagged with any of of github actions tag list ('github-actions', 'building-github-actions', 'github-actions-self-hosted-runners', 'github-actions-runners', 'github-actions-services', 'github-actions-artifacts', 'github-actions-reusable-workflows', 'github-actions-workflows', 'github-actions-marketplace')

In [31]:
import pandas as pd

In [47]:
posts_by_title = pd.read_json('data/posts_by_title.json')
posts_by_body = pd.read_json('data/posts_by_body.json')
posts_by_tags = pd.read_csv('data/posts_by_tags.csv')
posts_by_title.rename(columns={"CONCAT('https://stackoverflow.com/q/', p.Id)": 'link'}, inplace=True)
posts_by_body.rename(columns={"CONCAT('https://stackoverflow.com/q/', p.Id)": 'link'}, inplace=True)

In [48]:
posts_by_body = posts_by_body[posts_by_body['post_type_id']==1] # Keeping only post bodies that correspond to question bodies
posts_by_tags.drop(columns=['DeletionDate', 'OwnerDisplayName', 'LastEditorDisplayName', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate'], inplace=True);

In [49]:
posts_by_tags.rename(columns={'Id':'id', 'PostTypeId':'post_type_id', 'AcceptedAnswerId': 'accepted_answer_id', 'ParentId': 'parent_id', 'CreationDate':'creation_date',
                                'Score':'score', 'ViewCount':'view_count', 'Body':'post_body', 'OwnerUserId':'owner_user_id', 'LastEditorUserId':'last_editor_user_id',
                                'LastEditDate':'last_edit_date', 'LastActivityDate':'last_activity_date', 'Title':'post_title', 'Tags':'tags', 'AnswerCount':'answer_count',
                                'CommentCount':'comment_count', 'ContentLicense':'content_license', 'Unnamed: 23':'link'}, inplace=True)


In [59]:
#posts_by_tags.describe()
#posts_by_title.describe()
posts_by_body.describe()

,id,post_type_id,accepted_answer_id,parent_id,score,view_count,owner_user_id,last_editor_user_id,answer_count,comment_count
count,6.297000e+03,6297.0,2.327000e+03,0.0,6297.000000,6297.000000,6.255000e+03,2.837000e+03,6297.000000,6297.000000
mean,7.106578e+07,1.0,6.995068e+07,NaN,2.836748,2213.280769,8.619572e+06,6.780544e+06,0.978402,1.686835
std,5.347366e+06,0.0,5.549951e+06,NaN,12.542027,9546.348354,6.515283e+06,6.089129e+06,1.159121,2.453132
min,5.417676e+07,1.0,5.417763e+07,NaN,-3.000000,6.000000,9.100000e+01,-1.000000e+00,0.000000,0.000000
25%,6.738557e+07,1.0,6.578261e+07,NaN,0.000000,142.000000,2.628868e+06,1.778005e+06,0.000000,0.000000
50%,7.239494e+07,1.0,7.103011e+07,NaN,1.000000,457.000000,7.615872e+06,4.621513e+06,1.000000,1.000000
75%,7.561174e+07,1.0,7.481410e+07,NaN,2.000000,1350.000000,1.370580e+07,1.081728e+07,1.000000,3.000000
max,7.759154e+07,1.0,7.758567e+07,NaN,348.000000,321529.000000,2.302257e+07,2.290069e+07,35.000000,26.000000


In [65]:
df_union = pd.concat([posts_by_tags, posts_by_title, posts_by_body]).drop_duplicates().reset_index(drop=True)
df_union.describe()

,id,post_type_id,accepted_answer_id,parent_id,score,view_count,owner_user_id,last_editor_user_id,answer_count,comment_count
count,1.758200e+04,17582.0,6.643000e+03,0.0,17582.000000,17582.000000,1.745500e+04,7.731000e+03,17582.00000,17582.000000
mean,7.119717e+07,1.0,7.019367e+07,NaN,2.859629,2332.955295,8.661308e+06,6.650777e+06,0.99636,1.668297
std,5.244839e+06,0.0,5.474512e+06,NaN,12.111609,9413.060583,6.520149e+06,6.033402e+06,1.13754,2.416215
min,5.417676e+07,1.0,5.417763e+07,NaN,-6.000000,6.000000,9.100000e+01,-1.000000e+00,0.00000,0.000000
25%,6.776210e+07,1.0,6.613418e+07,NaN,0.000000,157.000000,2.628868e+06,1.688639e+06,0.00000,0.000000
50%,7.253435e+07,1.0,7.122424e+07,NaN,1.000000,503.000000,7.728038e+06,4.417586e+06,1.00000,1.000000
75%,7.564920e+07,1.0,7.497254e+07,NaN,2.000000,1473.000000,1.373346e+07,1.034897e+07,1.00000,2.000000
max,7.774041e+07,1.0,7.788818e+07,NaN,355.000000,337310.000000,2.317888e+07,2.316161e+07,35.00000,26.000000


In [64]:
df_merged_1 = pd.merge(posts_by_tags, posts_by_title, how='inner', on='id')
df_intersection = pd.merge(df_merged_1, posts_by_body, how='inner', on='id')
df_intersection.describe()

,id,post_type_id_x,accepted_answer_id_x,parent_id_x,score_x,view_count_x,owner_user_id_x,last_editor_user_id_x,answer_count_x,comment_count_x,...,comment_count_y,post_type_id,accepted_answer_id,parent_id,score,view_count,owner_user_id,last_editor_user_id,answer_count,comment_count
count,2.857000e+03,2857.0,1.146000e+03,0.0,2857.000000,2857.000000,2.836000e+03,1.257000e+03,2857.000000,2857.000000,...,2857.000000,2857.0,1.140000e+03,0.0,2857.000000,2857.000000,2.836000e+03,1.256000e+03,2857.000000,2857.000000
mean,7.041748e+07,1.0,6.942101e+07,NaN,4.052503,3139.711936,8.541469e+06,6.970519e+06,1.083304,1.728036,...,1.728386,1.0,6.938052e+07,NaN,3.901995,2926.654533,8.541469e+06,6.957873e+06,1.070004,1.728386
std,5.605049e+06,0.0,5.799237e+06,NaN,16.773162,12653.223765,6.387419e+06,6.085586e+06,1.387491,2.492825,...,2.493214,0.0,5.787363e+06,NaN,16.259177,11920.330407,6.387419e+06,6.071022e+06,1.381652,2.493214
min,5.417676e+07,1.0,5.417763e+07,NaN,-2.000000,8.000000,9.100000e+01,-1.000000e+00,0.000000,0.000000,...,0.000000,1.0,5.417763e+07,NaN,-2.000000,6.000000,9.100000e+01,-1.000000e+00,0.000000,0.000000
25%,6.630318e+07,1.0,6.472888e+07,NaN,0.000000,236.000000,2.628868e+06,2.031033e+06,0.000000,0.000000,...,0.000000,1.0,6.472536e+07,NaN,0.000000,194.000000,2.628868e+06,2.028900e+06,0.000000,0.000000
50%,7.175084e+07,1.0,7.047160e+07,NaN,1.000000,647.000000,7.615872e+06,4.904027e+06,1.000000,1.000000,...,1.000000,1.0,7.041612e+07,NaN,1.000000,587.000000,7.615872e+06,4.900238e+06,1.000000,1.000000
75%,7.540525e+07,1.0,7.454980e+07,NaN,3.000000,1829.000000,1.341090e+07,1.106587e+07,1.000000,3.000000,...,3.000000,1.0,7.450144e+07,NaN,3.000000,1666.000000,1.341090e+07,1.105684e+07,1.000000,3.000000
max,7.758694e+07,1.0,7.764939e+07,NaN,355.000000,337310.000000,2.302257e+07,2.302257e+07,35.000000,26.000000,...,26.000000,1.0,7.758567e+07,NaN,348.000000,321529.000000,2.302257e+07,2.261377e+07,35.000000,26.000000


### Number of questions filtered:
- posts_by_title: 4538
- posts_by_body: 6297
- posts_by_tags: 9828
- df_intersection: 2857
- df_union: 17582

In [75]:
df_intersection['tags'].str.contains('<github-actions>').value_counts()

tags
True     2848
False       9
Name: count, dtype: int64

In [77]:
df_intersection.columns

Index(['id', 'post_type_id_x', 'accepted_answer_id_x', 'parent_id_x',
       'creation_date_x', 'score_x', 'view_count_x', 'post_body_x',
       'owner_user_id_x', 'last_editor_user_id_x', 'last_edit_date_x',
       'last_activity_date_x', 'post_title_x', 'tags_x', 'answer_count_x',
       'comment_count_x', 'content_license_x', 'link_x', 'post_type_id_y',
       'accepted_answer_id_y', 'parent_id_y', 'creation_date_y', 'score_y',
       'view_count_y', 'post_body_y', 'owner_user_id_y',
       'last_editor_user_id_y', 'last_edit_date_y', 'last_activity_date_y',
       'post_title_y', 'tags_y', 'answer_count_y', 'comment_count_y',
       'content_license_y', 'link_y', 'post_type_id', 'accepted_answer_id',
       'parent_id', 'creation_date', 'score', 'view_count', 'post_body',
       'owner_user_id', 'last_editor_user_id', 'last_edit_date',
       'last_activity_date', 'post_title', 'tags', 'answer_count',
       'comment_count', 'content_license', 'link'],
      dtype='object')

In [76]:
df_intersection

,id,post_type_id_x,accepted_answer_id_x,parent_id_x,creation_date_x,score_x,view_count_x,post_body_x,owner_user_id_x,last_editor_user_id_x,...,owner_user_id,last_editor_user_id,last_edit_date,last_activity_date,post_title,tags,answer_count,comment_count,content_license,link
0,54176763,1,54177628.0,NaN,2019-01-14 06:40:49,0,808,"<p>I'm trying out Filters in GitHub Actions, h...",7110144.0,1421222.0,...,7110144.0,1421222.0,2019-02-06 19:49:10,2019-02-06 19:49:10,"GitHub Actions: Filter returns ""jq: error Cann...",<github><github-actions>,1.0,0,CC BY-SA 4.0,https://stackoverflow.com/q/54176763
1,54310050,1,60067489.0,NaN,2019-01-22 14:06:45,62,56073,<p>My use case is I want to have a unique vers...,774437.0,1421222.0,...,774437.0,1421222.0,2019-02-06 19:45:45,2023-09-26 17:25:18,How to version build artifacts using GitHub Ac...,<github><automation><versioning><continuous-de...,7.0,3,CC BY-SA 4.0,https://stackoverflow.com/q/54310050
2,54483260,1,54492739.0,NaN,2019-02-01 16:18:10,16,10307,<p>I woke up to my GitHub Actions BETA invite ...,1418711.0,1421222.0,...,1418711.0,1421222.0,2019-02-06 19:44:44,2023-09-11 11:02:11,Is it possible to persist a WORKDIR between Ac...,<github><github-actions>,4.0,1,CC BY-SA 4.0,https://stackoverflow.com/q/54483260
3,55110729,1,57958803.0,NaN,2019-03-11 21:32:03,82,61091,<p>Say I have a GitHub actions workflow with 2...,4449679.0,226526.0,...,4449679.0,226526.0,2019-03-11 23:35:57,2022-08-13 11:27:44,How do I cache steps in GitHub actions?,<github><github-actions>,6.0,3,CC BY-SA 4.0,https://stackoverflow.com/q/55110729
4,56030316,1,NaN,NaN,2019-05-07 20:51:08,5,1750,"<p>So far, I was under the impression as per t...",3403196.0,3403196.0,...,3403196.0,3403196.0,2019-05-07 20:59:16,2019-08-23 12:15:33,does /github/home persist between github actions?,<github><github-actions>,2.0,1,CC BY-SA 4.0,https://stackoverflow.com/q/56030316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2852,77564875,1,NaN,NaN,2023-11-28 14:55:03,0,45,<p>I have a private repo which has all GitHub ...,14900339.0,3001761.0,...,14900339.0,3001761.0,2023-11-28 16:29:42,2023-11-28 16:29:42,Unable to call private repo in GitHub Actions,<github><github-actions>,1.0,0,CC BY-SA 4.0,https://stackoverflow.com/q/77564875
2853,77566308,1,NaN,NaN,2023-11-28 18:27:14,1,143,"<p>I have a GitHub repository where I build, p...",19488382.0,2123530.0,...,19488382.0,2123530.0,2023-11-29 22:34:55,2023-11-29 22:34:55,Github Actions logs says .env: permission deni...,<git><docker><github><github-actions>,1.0,3,CC BY-SA 4.0,https://stackoverflow.com/q/77566308
2854,77572403,1,77585666.0,NaN,2023-11-29 15:17:24,1,85,<p>I have a private npm package inside an orga...,13203366.0,3001761.0,...,13203366.0,3001761.0,2023-11-29 18:04:03,2023-12-01 13:40:00,Unable to install a private npm package inside...,<node.js><github><npm><github-actions>,1.0,2,CC BY-SA 4.0,https://stackoverflow.com/q/77572403
2855,77585640,1,NaN,NaN,2023-12-01 13:35:34,0,44,<p>I have a git repository where some JUnit-Te...,519334.0,NaN,...,519334.0,NaN,None,2023-12-01 13:35:34,Github Actions JUnittest Failing: How to get t...,<java><gradle><junit><github-actions><error-lo...,0.0,2,CC BY-SA 4.0,https://stackoverflow.com/q/77585640
